In [1]:
# Импортируем необходимые библиотеки:
# pandas для манипуляции данными,
# scikit-surprise для функциональности рекомендательных систем, такой как чтение датасетов и применение алгоритмов.
import pandas as pd
from surprise import Dataset, Reader, SVD

# Загружаем датасет с рейтингами фильмов.
ratings = pd.read_csv("ratings.csv")

# Создаем таблицу сводки, чтобы увидеть рейтинги, которые пользователи ставят каждому фильму.
# Индекс: userId, Колонки: movieId, Значения: рейтинг.
user_movie_rating = ratings.pivot_table(index='userId', columns='movieId', values='rating')
# Показываем первые 100 строк для быстрого просмотра.
user_movie_rating.head(100)  

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Определяем объект Reader, указывая шкалу рейтинга.
reader = Reader(rating_scale=(0.5, 5))  # Рейтинги фильмов варьируются от 0.5 до 5.

# Загружаем данные из DataFrame.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Строим полный тренировочный набор данных.
trainset = data.build_full_trainset()

# Создаем экземпляр алгоритма SVD.
# Если алгоритму передан аргумент biased=true (по умолчанию true),
# Scikit-compose реализует Знаменитый алгоритм SVD, популяризированный Саймоном Фанком во время премии Netflix.
# Если biased=false, то будет использован Probabilistic Matrix Factorization, т.е. SVD без весов.
algorithm = SVD(biased=False)
 # Обучаем модель на тренировочном наборе.
algorithm.fit(trainset) 

# Подготавливаем пустой список для тестового набора данных.
testset = []

# Проходим по всем пользователям и фильмам для создания тестового набора.
for uid in trainset.all_users():
    for iid in trainset.all_items():
        # Проверяем отсутствие пары (uid, iid) в тренировочном наборе.
        if not trainset.ur.get(uid) or (iid not in map(lambda x:x[0], trainset.ur[uid])):
            # Добавляем в тестовый набор с None в качестве фактического рейтинга.
            testset.append((trainset.to_raw_uid(uid), trainset.to_raw_iid(iid), None))

# Получаем предсказания для отсутствующих значений.
predictions = algorithm.test(testset)

# Подготавливаем список для хранения предсказанных рейтингов.
pred_ratings = []
for pred in predictions:
    pred_ratings.append([pred.uid, pred.iid, pred.est])

In [3]:
# Преобразуем список предсказаний в DataFrame.
pred_df = pd.DataFrame(pred_ratings, columns=['userId', 'movieId', 'rating'])

# Объединяем предсказанные рейтинги с исходными рейтингами.
complete_ratings = pd.concat([ratings[['userId', 'movieId', 'rating']], pred_df])

# Создаем итоговую таблицу сводки с полными данными, включающую как фактические, так и предсказанные рейтинги.
complete_user_movie_rating = complete_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Сортируем столбцы итоговой сводной таблицы согласно исходной таблице сводки рейтингов.
complete_user_movie_rating = complete_user_movie_rating[user_movie_rating.columns]

# Показываем первые 100 строк итоговой таблицы для быстрого просмотра.
complete_user_movie_rating.head(100)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.023892,4.000000,3.195747,2.965501,4.000000,3.480668,3.324325,3.229843,3.832203,...,2.302934,1.437789,2.273051,2.178198,1.772275,2.044253,1.711440,1.847448,1.768108,2.278554
2,3.805124,3.391686,2.847985,2.381501,2.496465,3.653650,2.779703,2.458665,2.764377,3.497063,...,2.065170,1.170557,2.121437,1.833653,1.520911,1.862570,1.691109,1.626996,1.630659,1.723862
3,2.010279,1.690943,1.941270,0.918662,1.362216,2.366561,1.655605,0.983284,1.720312,2.626576,...,0.810077,0.740962,0.641339,0.674648,0.851640,0.993173,0.768462,0.600273,0.552426,0.845430
4,3.313453,3.407687,2.548222,2.155697,2.358487,3.393890,1.973636,2.312940,2.818612,2.126390,...,1.085095,1.433650,1.509518,1.928051,0.783197,1.212397,1.380172,1.028491,1.367628,2.570300
5,4.000000,3.636649,2.829283,2.023012,2.707186,3.422504,2.926238,2.402048,2.589383,3.007838,...,1.619312,1.002653,1.426588,1.534813,1.198466,1.531349,1.635173,1.502232,1.179926,1.589133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.000000,3.775823,3.396784,2.460284,2.656240,4.174583,2.757804,2.377825,2.685170,3.463263,...,1.646124,1.066229,1.794956,1.460752,1.537775,1.724446,1.157764,1.885554,1.316762,1.679574
97,4.101319,3.722966,3.483721,2.464153,2.990837,4.355678,2.955249,2.838065,2.921633,3.892687,...,2.016914,1.105220,2.032328,1.943184,1.377164,1.971496,1.613279,1.939217,1.512306,2.234951
98,4.500000,3.693271,3.369709,2.459442,2.541924,4.052561,2.810469,2.459302,3.103271,3.719011,...,1.832312,1.206907,1.970018,2.108308,1.569274,1.962627,1.800030,1.753021,1.658146,2.003844


In [4]:
# Экспортируем итоговую таблицу в CSV файл.
complete_user_movie_rating.to_csv("predicted_ratings.csv")